In [1]:
import matplotlib.pyplot as plt
%matplotlib qt
from tistools import *
from pprint import pprint

In [2]:
inputfile = "./retis.rst"
indir = "./"
folders = glob.glob("./0[0-9][0-9]")
folders = sorted(folders)
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol[len(indir):])
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one = True
        pe.set_in_zero_minus_one = True
    if i==1:
        pe.set_in_zero_plus = True 
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)
    pe.set_orders(load=True, acc_only=True, save=True)
    if i > 1:
        print("Calculating first hitting lengths to middle interface")
        set_tau_first_hit_M_distrib(pe)
        print("Done.")
        print("Calculating path lengths.")
        set_tau_distrib(pe)
        print("Done.")

################################################################################
./000
pathensemble info: 
{'cyclenumbers': array([     0,      1,      2, ...,  99998,  99999, 100000]),
 'flags': array(['ACC', 'ACC', 'ACC', ..., 'FTL', 'BTL', 'FTL'], dtype='<U3'),
 'generation': array(['ld', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': False,
 'in_zero_plus': False,
 'interfaces': [[0.1, 0.15000000000000002, 0.2],
                ['l_[-1]', '( l_[-1] + l_[0] ) / 2', 'l_[0]']],
 'lambmaxs': array([0.18995829, 0.20115691, 0.20129873, ..., 0.16037035, 0.18323515,
       0.17180941]),
 'lambmins': array([0.08995829, 0.17135828, 0.0973619 , ..., 0.09888888, 0.12286248,
       0.09614375]),
 'lengths': array([  4,  69, 104, ...,  81,  62,  67]),
 'lmrs': array(['LM*', 'R*R', 'LMR', ..., 'LM*', '*M*', 'LM*'], dtype='<U3'),
 'name': '000',
 'ncycle': 100001,
 'newpathnumbers': array([    0,     1,     2, ..., 44278, 44278, 44278]),
 'orders'

In [3]:
# Analyze the pptis simulation.
# Analysis output is saved to the data dictionary.
data = {}
for i, pe in enumerate(pathensembles):
    print("doing pathensemble {}".format(i))
    if i == 0:
        continue  # TODO: make [0-] analysis. 
    masks = get_lmr_masks(pe)
    loadmask = get_generation_mask(pe, "ld")
    print("Amount of loads: {}".format(np.sum(loadmask)))
    #hardloadmask = get_hard_load_mask(loadmask)
    accmask = get_flag_mask(pe, "ACC")
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
                (pe.lmrs == ptype).astype(int), pe.weights)
    # running average analysis
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    # you'll still have to hardload select pe.weigths... TODO
    for (ptype, p_loc) in zip(pathtypes, 
                              running_avg_local_probs(pathtype_cycles, 
                                                      pe.weights, tr = False)):
        data[i]["running"]["plocal"][ptype] = p_loc
    plocfull = get_local_probs(pe, tr=False)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]
    
    upe = pe.unify_pe()
    makefigs = True 
    if makefigs:
        # Cross distances distribution
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
        fig,ax =plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution
        data[i]["pathlengths"] = pathlength_distr(upe)
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)


doing pathensemble 0
doing pathensemble 1
Amount of loads: 1
Weights of the different paths:
wRMR = 0
wRML = 8268
wLMR = 8311
wLML = 83421
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.0906008808267562
pLML = 0.9093991191732438
Are all weights 1?  True
Are all paths accepted?  True


/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype

doing pathensemble 2
Amount of loads: 1
Weights of the different paths:
wRMR = 167
wRML = 4373
wLMR = 4262
wLML = 91198
Local crossing probabilities:
pRMR = 0.03678414096916299
pRML = 0.963215859030837
pLMR = 0.044646972553949296
pLML = 0.9553530274460507
Are all weights 1?  True
Are all paths accepted?  True


/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])


doing pathensemble 3
Amount of loads: 1
Weights of the different paths:
wRMR = 12079
wRML = 14170
wLMR = 13789
wLML = 59960
Local crossing probabilities:
pRMR = 0.4601699112347137
pRML = 0.5398300887652863
pLMR = 0.18697202673934563
pLML = 0.8130279732606543
Are all weights 1?  True
Are all paths accepted?  True


/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])


doing pathensemble 4
Amount of loads: 1
Weights of the different paths:
wRMR = 14332
wRML = 27536
wLMR = 27498
wLML = 30633
Local crossing probabilities:
pRMR = 0.3423139390465272
pRML = 0.6576860609534728
pLMR = 0.4730350415440987
pLML = 0.5269649584559013
Are all weights 1?  True
Are all paths accepted?  True


/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])


doing pathensemble 5
Amount of loads: 1
Weights of the different paths:
wRMR = 9372
wRML = 18027
wLMR = 18380
wLML = 54221
Local crossing probabilities:
pRMR = 0.34205627942625644
pRML = 0.6579437205737436
pLMR = 0.25316455696202533
pLML = 0.7468354430379747
Are all weights 1?  True
Are all paths accepted?  True


/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:553: RuntimeWarning: invalid value encountered in divide
  p_NP = cumsums['LMR']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:554: RuntimeWarning: invalid value encountered in divide
  p_NN = cumsums['LML']/(cumsums['LMR']+cumsums['LML'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:555: RuntimeWarning: invalid value encountered in divide
  p_PP = cumsums['RMR']/(cumsums['RML']+cumsums['RMR'])
/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/tistools/repptis_analysis.py:556: RuntimeWarning: invalid value encountered in divide
  p_PN = cumsums['RML']/(cumsums['RML']+cumsums['RMR'])


In [4]:
# Global crossing probabilities
# WITHOUT ERRORS #  
# Full data 
psfull = []
for i in range(1, len(pathensembles)):
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})
    
Pminfull, Pplusfull, Pcrossfull = get_globall_probs(psfull)

# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(Pcrossfull, "o", c = "r")
ax.set_xlabel("intf")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")
print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
print("which is the case!")
print("Here, the load immediately disappeared. For a simulation where this is")
print("not the case, the above code should be adapted a little bit.")

This should be the same as the repptis_report.pdf value: 0.00020540578383777418
which is the case!
Here, the load immediately disappeared. For a simulation where this is
not the case, the above code should be adapted a little bit.


In [5]:
# Or we can use the get_global_probz function, using lists of the local probs
pmps = [data[i]["full"]["LMR"] for i in range(1,len(pathensembles))]
pmms = [data[i]["full"]["LML"] for i in range(1,len(pathensembles))]
ppps = [data[i]["full"]["RMR"] for i in range(1,len(pathensembles))]
ppms = [data[i]["full"]["RML"] for i in range(1,len(pathensembles))]
a,b,c = get_global_probz(pmps, pmms, ppps, ppms)
print("This should be the same as the repptis_report.pdf value:", c[-1])

# for MSM we only want the body ensembles: 
# but first, we only need the body ensembles
pmps = pmps[1:]
ppps = ppps[1:]
ppms = ppms[1:]
pmms = pmms[1:]

This should be the same as the repptis_report.pdf value: 0.00020540578383777418


In [6]:
# Let's build the MSM. 

# we need a mapping from 2D space (k,l) with k,l in {-1,1} to 1D space (0, 1, 2, 3)
def plocify(pmm, pmp, ppm, ppp):
    ploc = np.zeros((3,3), dtype=float)
    ploc[-1,-1] = pmm
    ploc[-1,1] = pmp
    ploc[1,-1] = ppm
    ploc[1,1] = ppp
    assert pmm + pmp == 1.0, f"pmm ({pmm}) + pmp ({pmp}) != 1.0"
    assert ppm + ppp == 1.0, f"ppm ({ppm}) + ppp ({ppp}) != 1.0"
    return ploc

def pathlengthify(taumm, taump, taupm, taupp):
    tau = np.zeros((3,3), dtype=float)
    tau[-1,-1] = taumm
    tau[-1,1] = taump
    tau[1,-1] = taupm
    tau[1,1] = taupp
    return tau

def kl_to_idx(i,j):
    """
    i,j in {-1,1}
    k in {0,1,2,3}
    --------------
    (-1, -1) -> 0
    (-1,  1) -> 1
    ( 1, -1) -> 2
    ( 1,  1) -> 3
    """
    assert i in {-1,1} and j in {-1,1}, f"i ({i}) and j ({j}) should be in {-1,1}"
    if i == -1:
        return 0 if j == -1 else 1
    return 2 if j == -1 else 3

def ikl_to_n(i,k,l):
    return 4*i + kl_to_idx(k,l)


pmp0plus = data[1]["full"]["LMR"]
# define plocs 
plocs = []
Nbodies = len(interfaces)-2
for i in range(Nbodies):
    plocs.append(plocify(pmms[i], pmps[i], ppms[i], ppps[i]))
    
# now we can go on to defining the transition matrix M 
# at first, we will include S_{-1}^{-1,-1} and S_{N-2}^{1,1}, and afterwards 
# we will remove them.
M = np.zeros((4*(Nbodies), 4*(Nbodies)), dtype=float)

# we know that FROM i, k, l we can go to i+l, -l, +1 or i+l, -l, -1
# For i = 0 and i = N-3, we will do it manually
# So we can fill in the matrix row by row.
for i in range(0, Nbodies):
    for k in (-1,1):
        for l in (-1,1):
            # we are in state i, k, l
            # we can go to state i+l, -l, +1 or i+l, -l, -1
            #print(f"We are in state ({i}, {k}, {l})")
            if (i != 0 and i != Nbodies-1) or\
                    (i == 0 and l == 1) or\
                    (i == Nbodies-1 and l == -1):
                for l2 in {-1,1}:
                    #print(f"---And go to state ({i+l}, {-l}, {l2})")
                    M[ikl_to_n(i,k,l), ikl_to_n(i+l,-l,l2)] =\
                        plocs[i+l][-l,l2]

# Set absorbing states in (0, -1, -1), (0, 1, -1), (N-3, -1, 1), (N-3, 1, 1)
M[0,0] = 1
M[2,2] = 1
M[-1,-1] = 1
M[-3,-3] = 1

In [7]:
# remove rows 0, 2, -1 and -3, aslo remove columns 0, 2, -1 and -3.
print("Solving the linear system with P_A = 0 and P_B = 1 hardcoded.")
Mhit = np.delete(M, [0,2,-1,-3], axis=0)
Mhit = np.delete(Mhit, [0,2,-1,-3], axis=1)

# P(N-2, -1, +1) == 1, add contribution C vector for this.
C = np.zeros(len(Mhit))
#C[2] = plocs[0][+1,-1]  # P(1, +1, -1) hits state A, so == 0
#C[4] = plocs[0][+1,-1]
C[-3] = plocs[-1][-1,+1]
C[-5] = plocs[-1][-1,+1]

# solution for the hitting probability vector
sol = np.linalg.inv(np.eye(len(Mhit)) - Mhit) @ C
# multiply by pmp0plus and plocs[0][-1][+1] and select the first element
# (aka, the element starting from P(1, -1, +1))
Pglobalcross = pmp0plus * plocs[0][-1][+1] * sol[0]
print("This should be the same as the repptis_report.pdf value:",
      Pglobalcross)
print("Which is the case.")

Solving the linear system with P_A = 0 and P_B = 1 hardcoded.
This should be the same as the repptis_report.pdf value: 0.00020540578383777418
Which is the case.


In [8]:
# Let's try again now by just calcing the eigvals and eigvecs of M
print("Just taking the lambda=1 eigenvector of M gives different results:")
evals, evecs = np.linalg.eig(M)
print("% diff:",(100*np.min(np.abs(0.00020540578383777418-pmp0plus*plocs[0][-1,1]*evecs)))/0.00020540578383777418)
evals, evecs = np.linalg.eig(M[1:-1,1:-1])
print("% diff:",(100*np.min(np.abs(0.00020540578383777418-pmp0plus*plocs[0][-1,1]*evecs)))/0.00020540578383777418)
print("It does not work this way")

Just taking the lambda=1 eigenvector of M gives different results:
% diff: 7.182370446262876
% diff: 10.777627368737152
It does not work this way


In [9]:
print("Using msmtools will also work for hitting probs.")
# Let's try with msmtools, this should definitely work as well.
# I already know I can't solve the MFPT question with this, but 
# this might come in handy whenever we DO include multidimensionality
from msmtools.analysis import (is_transition_matrix, is_connected, mfpt, 
                               stationary_distribution, is_reversible)
print("Using transition matrix with disconnected states: ")
print("is_tmat: ", is_transition_matrix(M))
print("is_connected: ", is_connected(M))
print("is_reversible: ", is_reversible(M))
print("And cutting away the first and last row and column: ")
print("is_tmat: ", is_transition_matrix(M[1:-1,1:-1]))
print("is_connected: ", is_connected(M[1:-1,1:-1]))
print("is_reversible: ", is_reversible(M[1:-1,1:-1]))
print("Still disconnected, as we have absorbing states for those hitting state A and B!")
print("Thus, here, we will just work with full M matrix.")
print("Nevermind, we need it to be W E A K L Y connected, so using M[1:-1,1:-1].")

Using msmtools will also work for hitting probs.
Using transition matrix with disconnected states: 
is_tmat:  True
is_connected:  False
is_reversible:  True
And cutting away the first and last row and column: 
is_tmat:  True
is_connected:  False
is_reversible:  True
Still disconnected, as we have absorbing states for those hitting state A and B!
Thus, here, we will just work with full M matrix.
Nevermind, we need it to be W E A K L Y connected, so using M[1:-1,1:-1].


In [10]:
# define states A and B
A = [1]
B = [-2]
# Get the committtor probabilities
from msmtools.analysis import committor
from msmtools.flux import tpt
test = tpt(M[1:-1,1:-1], A, B)
print("Same value?", (pmp0plus * plocs[0][-1][1] * test.committor)[0])
print("Real value:", 0.00020540578383777418)
print("So this works as expected")

Same value? 0.00020540578383777415
Real value: 0.00020540578383777418
So this works as expected


/home/wouter/miniconda3/envs/msming/lib/python3.8/site-packages/msmtools/flux/dense/tpt.py:265: RuntimeWarning: invalid value encountered in double_scalars
  kAB = totflux / (pi * qminus).sum()


In [13]:
tauvals = []
for i in range(2, Nbodies+2):
    tauLML = pathensembles[i].tauavg['LML']
    tauLMR = pathensembles[i].tauavg['LMR']
    tauRML = pathensembles[i].tauavg['RML']
    tauRMR = pathensembles[i].tauavg['RMR']
    tauvals.append(pathlengthify(tauLML, tauLMR, tauRML, tauRMR))

for i, tauval in enumerate(tauvals):
    print(f"For ensemble [{i+1}^+-] we have the amazing taus:\n", tauval[1:,1:])

For ensemble [1^+-] we have the amazing taus:
 [[1067.88023952  695.83238052]
 [ 711.85851713  559.16546415]]
For ensemble [2^+-] we have the amazing taus:
 [[1220.44862985 1134.34333098]
 [1130.4901008  1141.20280187]]
For ensemble [3^+-] we have the amazing taus:
 [[228.57828635 293.65347182]
 [289.53498436 442.28283224]]
For ensemble [4^+-] we have the amazing taus:
 [[739.35606061 473.00305098]
 [458.59717084 272.67733904]]


In [52]:
taus = np.zeros(4*Nbodies, dtype=float)
for i in range(2,Nbodies+2):
    print("doing", (4*(i-2)), "up to", (4*(i-2)+3))
    if i>2:
        taus[4*(i-2)] = pathensembles[i].tauavg["LML"] - pathensembles[i].tau1avg["LML"] + 1
        taus[4*(i-2)+1] = pathensembles[i].tauavg["LMR"] - pathensembles[i].tau1avg["LMR"] + 1
    else: 
        taus[4*(i-2)] = pathensembles[i].tauavg["LML"] - 1
        taus[4*(i-2)+1] = pathensembles[i].tauavg["LMR"] - 1
    taus[4*(i-2)+2] = pathensembles[i].tauavg["RML"] - pathensembles[i].tau1avg["RML"] + 1
    taus[4*(i-2)+3] = pathensembles[i].tauavg["RMR"] - pathensembles[i].tau1avg["RMR"] + 1


doing 0 up to 3
doing 4 up to 7
doing 8 up to 11
doing 12 up to 15


In [53]:
taus

array([ 558.16546415,  710.85851713,  369.53898925,  676.53293413,
        806.64019346,  820.79788237,  950.6037403 , 1054.17244805,
        263.80641792,  124.23616263,  252.67413568,  185.86289422,
        230.33682522,  418.43117519,  274.52687635,  541.36182245])

In [54]:
Mhit = M[1:-1,1:-1]
Mhit[1,1] = 0
Mhit[-2,-2] = 0
tauhit = taus[1:-1]
sol = np.linalg.inv(np.eye(len(Mhit)) - Mhit) @ tauhit

In [55]:
print([Mhit[i,i]for i in range(len(Mhit))])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [56]:
print(sol)

[2341.76510387  369.53898925 2307.43952087 1247.46318905 3298.26942126
 1391.42673589 3531.64398694 2640.09673624 2296.30572062 2628.964454
 2357.93245222 2766.52324708  418.43117519 2810.71329822]


In [97]:
# Now we leave msmtools, and use the matrix equations to look at MFPTS
# first, we must create the \tau vector, which is the average pathlength 
# for each state. 

Mhit = np.delete(M, [0,2,-1,-3], axis=0)
Mhit = np.delete(Mhit, [0,2,-1,-3], axis=1)
tauhit = np.delete(taus, [0,2,-1,-3])

# add contribution for the absorbing states
C = np.zeros(len(Mhit))
C[2] = taus[2]
C[4] = taus[2]
C[-3] = taus[-3]
C[-5] = taus[-3]

# solution for the hitting probability vector
sol = np.linalg.inv(np.eye(len(Mhit)) - Mhit) @ Mhit @ (tauhit + C)
# multiply by pmp0plus and plocs[0][-1][+1] and select the first element
# (aka, the element starting from P(1, -1, +1))
print(sol)
print("This should be the same as the repptis_report.pdf value:",
      Pglobalcross)
print("Which is the case.")


[1700.70579334 1700.70579334   87.37111619 2800.22566036   87.37111619
 2800.22566036 2530.41669951 2270.47322171 2530.41669951 2270.47322171
 2811.78833606 2811.78833606]
This should be the same as the repptis_report.pdf value: 0.00020540578383777418
Which is the case.


In [44]:
sol = Mhit @ tauhit @ np.linalg.inv(np.eye(len(Mhit)) - Mhit)

In [23]:
pmp0plus = data[1]["full"]["LMR"]
pmm0plus = data[1]["full"]["LML"]

In [57]:
tau_0plus = pmm0plus * (data[1]["pathlengths"]["LML"]["mean"] - 2) +\
            pmp0plus * (plocs[0][-1,-1] * (tauvals[0][-1,-1]-2) +\
                        plocs[0][-1, 1] * (sol[0]))
print("tau_0plus:", tau_0plus)
print("realvalue:", 102.64497000000001, "with std.dev. 295.51")
print("This is really good!")

tau_0plus: 96.57247721281611
realvalue: 102.64497000000001 with std.dev. 295.51


In [89]:
# Just checking some path-chains and their probabilities.
path1 = tauvals[1][-1, 1] + (tauvals[2][-1,-1]-pathensembles[3].tau1avg["LML"]) + (tauvals[1][1,-1]-pathensembles[2].tau1avg["RML"])
prob_path1 = plocs[1][-1,-1]*plocs[0][1,-1]
path2 = tauvals[1][-1, 1] + (tauvals[2][-1,+1]-pathensembles[3].tau1avg["LMR"]) + (tauvals[3][-1,-1]-pathensembles[4].tau1avg["LML"]) +\
        (tauvals[2][1,-1]-pathensembles[3].tau1avg["RML"]) + (tauvals[1][1,-1]-pathensembles[2].tau1avg["RML"])
prob_path2 = plocs[1][-1,1] * plocs[2][-1, -1] * plocs[1][1,-1] * plocs[0][1,-1]
path3 = tauvals[1][-1, 1] + (tauvals[2][-1,-1]-pathensembles[3].tau1avg["LML"]) + (tauvals[1][1,1]-pathensembles[2].tau1avg["RMR"]) +\
        (tauvals[2][-1,-1]-pathensembles[3].tau1avg["LML"]) + (tauvals[1][1,-1]-pathensembles[2].tau1avg["RML"])
prob_path3 = plocs[1][-1,1] * plocs[2][-1, -1] * plocs[1][1,1] * plocs[2][-1,-1] * plocs[1][1,-1] * plocs[0][1,-1]
path4 = tauvals[1][-1, 1] + (tauvals[2][-1,+1]-pathensembles[3].tau1avg["LMR"]) + (tauvals[3][-1,+1]-pathensembles[4].tau1avg["LMR"]) +\
        (tauvals[4][-1,1] - pathensembles[5].tau1avg["LMR"])
prob_path4 = plocs[1][-1,1] * plocs[2][-1, 1] * plocs[3][-1,1]
path5 = tauvals[1][-1, 1] + (tauvals[2][-1,+1]-pathensembles[3].tau1avg["LMR"]) + (tauvals[3][-1,+1]-pathensembles[4].tau1avg["LMR"]) +\
        (tauvals[4][-1,-1] - pathensembles[5].tau1avg["LML"]) + (tauvals[3][1,-1] - pathensembles[4].tau1avg["RML"]) +\
        (tauvals[2][1,-1] - pathensembles[3].tau1avg["RML"]) + (tauvals[1][1,-1] - pathensembles[2].tau1avg["RML"]) +\
        (tauvals[1][1,-1] - pathensembles[2].tau1avg["RML"])
prob_path5 = plocs[1][-1,1] * plocs[2][-1, 1] * plocs[3][-1,-1]

In [90]:
print(path1)
print(path2)
print(path3)
print(path4)
print(path5)
print(prob_path1)
print(prob_path2)
print(prob_path3)
print(prob_path4)
print(prob_path5)

1880.0376998426466
3102.605546962064
3357.210827436691
2064.3237373185475
3806.585241821238
0.7831214376803615
0.05123173744090692
0.012423359130139637
0.022390967199045746
0.06605335323718495


In [91]:
print(prob_path1 + prob_path2 + prob_path3 + prob_path4 + prob_path5)
print(prob_path1*path1 + prob_path2*path2 + prob_path3*path3 + prob_path4*path4 + prob_path5*path5)

0.9352208546876387